<a href="https://colab.research.google.com/github/napatswift/vc/blob/main/vote62/table-localization/mmdet-train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install openmim
!mim install mmcv-full
!pip install gradio

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .

In [ ]:
import mmdet
print(mmdet.__version__)
# Example output: 2.23.0

In [ ]:
!mim download mmdet \
  --config yolov3_mobilenetv2_320_300e_coco \
  --dest .

In [ ]:
!wget https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth -P checkpoints/

In [ ]:
!wget https://github.com/napatswift/vc/releases/download/table-det-v-610i/table-det-610.tar.gz

In [ ]:
!tar zxf table-det-610.tar.gz

In [ ]:
with open('configs/vote-config.py', 'w') as fp:
  fp.write("""_base_ = 'mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_1x_coco.py'

# We also need to change the num_classes in head to match the dataset's annotation
model = dict(
    roi_head=dict(
        bbox_head=dict(num_classes=1),
        mask_head=dict(num_classes=1)))

# Modify dataset related settings
dataset_type = 'COCODataset'
classes = ('Table',)
data = dict(
    train=dict(
        classes=classes,
        img_prefix='data/table-det-610/',
        ann_file='data/table-det-610/train_coco.json'),
    val=dict(
        classes=classes,
        img_prefix='data/table-det-610/',
        ann_file='data/table-det-610/test_coco.json'),
    test=dict(
        classes=classes,
        img_prefix='data/table-det-610/',
        ann_file='data/table-det-610/test_coco.json'))

train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='Translate',),
    dict(type='Rotate',level=40,),
    dict(type='Shear',prob=0.6,),
    dict(type='ColorTransform',prob=0.7),
    dict(type='BrightnessTransform',prob=0.7),
    dict(type='Normalize', **dict(
      mean=[103.530, 116.280, 123.675], std=[1.0, 1.0, 1.0], to_rgb=False)),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks']),
]
optimizer = dict(lr=1e-3)
# We can use the pre-trained Mask RCNN model to obtain higher performance
load_from = 'checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'
""")

In [ ]:
!python \
  tools/train.py \
  configs/vote-config.py

In [ ]:
!python \
  tools/test.py\
  configs/vote-config.py work_dirs/vote-config/latest.pth --eval bbox segm

In [ ]:
from mmdet.apis import init_detector, inference_detector
import mmcv

In [ ]:
config_file = '/content/mmdetection/work_dirs/vote-config/vote-config.py'
checkpoint_file = '/content/mmdetection/work_dirs/vote-config/latest.pth'
model = init_detector(config_file, checkpoint_file, device='cuda:0')

In [ ]:
!tar -zcvf table-detector.tar.gz work_dirs/vote-config/vote-config.py work_dirs/vote-config/latest.pth

In [ ]:
import gradio as gr
def run(image):
  result = inference_detector(model, image)
  return model.show_result(image, result,)

gr.Interface(fn=run, inputs="image", outputs="image").launch()
